# MTG Card Pricing Analysis by Type

This notebook explores the pricing dataset joined with card data to provide comprehensive statistics by card type. We'll analyze pricing patterns across different card types (creatures, enchantments, instants, etc.) while excluding basic lands.

## Analysis Overview
- **Data Source**: CosmosDB with 110,000+ cards and pricing data  
- **Focus**: Statistical analysis by card type
- **Exclusions**: Basic lands  
- **Metrics**: Count, Min, Max, Average, Median prices

## Import Required Libraries

Import pandas, numpy, and database connection libraries for data analysis and visualization.

In [2]:
import pandas as pd
import numpy as np
import os
import sys
import asyncio
from datetime import datetime, timezone
import warnings
warnings.filterwarnings('ignore')

# Add project paths for imports (notebooks folder)
sys.path.append('/workspaces/mtgecorec')
sys.path.append('/workspaces/mtgecorec/core')

# Import the simple core database driver
from core.data_engine.cosmos_driver import get_mongo_client, get_collection

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Using core cosmos_driver system")
print(f"Notebook location: /workspaces/mtgecorec/notebooks/")

Libraries imported successfully!
Pandas version: 2.3.3
NumPy version: 2.3.5
Using core cosmos_driver system
Notebook location: /workspaces/mtgecorec/notebooks/


## Database Connection & Configuration

Set up connection to CosmosDB using the existing configuration management system.

In [2]:
# Set up database connection using core system
# Get MongoDB client
client = get_mongo_client()

# Specify database and collection names
database_name = "mtgecorec" 

## Load Card and Pricing Datasets

Load card data from the `cards` collection and pricing data from the `card_pricing_daily` collection, then join them together.

In [3]:
# Load cards data
print("Loading cards from database...")
cards_collection = get_collection(client, database_name, "cards")
all_cards = list(cards_collection.find({}))
print(f"✅ Loaded {len(all_cards)} cards from 'cards' collection")

# Load pricing data  
print("Loading pricing data from database...")
pricing_collection = get_collection(client, database_name, "card_pricing_daily")
all_pricing = list(pricing_collection.find({}))
print(f"✅ Loaded {len(all_pricing)} pricing records from 'card_pricing_daily' collection")

# Convert to DataFrames
df_cards = pd.DataFrame(all_cards)
df_pricing = pd.DataFrame(all_pricing)

print(f"\nCards DataFrame shape: {df_cards.shape}")
print(f"Pricing DataFrame shape: {df_pricing.shape}")

print(f"\nCards columns: {list(df_cards.columns)}")
print(f"Pricing columns: {list(df_pricing.columns)}")

# Show sample data
print("\n=== SAMPLE CARDS DATA ===")
print(df_cards[['id', 'name', 'type_line'] if all(col in df_cards.columns for col in ['id', 'name', 'type_line']) else df_cards.columns[:3]].head(3))

print("\n=== SAMPLE PRICING DATA ===")
print(df_pricing.head(3))

Loading cards from database...
✅ Loaded 110031 cards from 'cards' collection
Loading pricing data from database...
✅ Loaded 321433 pricing records from 'card_pricing_daily' collection

Cards DataFrame shape: (110031, 91)
Pricing DataFrame shape: (321433, 17)

Cards columns: ['_id', 'object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id', 'arena_id', 'tcgplayer_id', 'cardmarket_id', 'name', 'lang', 'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image', 'image_status', 'image_uris', 'mana_cost', 'cmc', 'type_line', 'oracle_text', 'colors', 'color_identity', 'keywords', 'produced_mana', 'legalities', 'games', 'reserved', 'game_changer', 'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint', 'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri', 'set_search_uri', 'scryfall_set_uri', 'rulings_uri', 'prints_search_uri', 'collector_number', 'digital', 'rarity', 'card_back_id', 'artist', 'artist_ids', 'illustration_id', 'border_color', 'frame', 'full_art', 'textle

## Join Card Data with Pricing Data

Join the cards dataset with pricing data using `card_pricing_daily.card_uuid` → `cards.id`.

In [4]:
# Join cards with pricing data
print("Joining card data with pricing data...")
print(f"Join key - Cards: 'id', Pricing: 'scryfall_id'")

# Check join keys exist
if 'id' not in df_cards.columns:
    print("❌ ERROR: 'id' column not found in cards data")
    print(f"Available columns in cards: {list(df_cards.columns)}")
    
if 'card_uuid' not in df_pricing.columns:
    print("❌ ERROR: 'card_uuid' column not found in pricing data")  
    print(f"Available columns in pricing: {list(df_pricing.columns)}")

# Perform the join (inner join to only keep cards with pricing)
df = df_cards.merge(df_pricing, left_on='id', right_on='scryfall_id', how='inner', suffixes=('', '_pricing'))

print(f"\n✅ Join completed!")
print(f"Cards with pricing: {len(df):,}")
print(f"Cards without pricing: {len(df_cards) - len(df):,}")
print(f"Join success rate: {(len(df)/len(df_cards))*100:.1f}%")

print(f"\nJoined DataFrame shape: {df.shape}")
print(f"Columns after join: {len(df.columns)} total")

# Display first few rows to understand the joined structure  
df.head()

Joining card data with pricing data...
Join key - Cards: 'id', Pricing: 'card_uuid'

✅ Join completed!
Cards with pricing: 321,433
Cards without pricing: -211,402
Join success rate: 292.1%

Joined DataFrame shape: (321433, 108)
Columns after join: 108 total


,_id,object,id,oracle_id,multiverse_ids,mtgo_id,arena_id,tcgplayer_id,cardmarket_id,name,...,source,tcgplayer_id_pricing,cardmarket_id_pricing,date,timestamp,created_at,collected_at,price_value,currency,finish
0,68d04b6b8255b4901067c399,card,0000419b-0bba-4488-8f7a-6194544ce91e,b34bb2dc-c1af-4d77-b0b3-a0fb342a5fc6,[668564],129825.0,91829.0,558404.0,777725.0,Forest,...,scryfall_usd,558404.0,777725.0,2025-12-12,2025-12-12T22:43:07.906282+00:00,2025-12-12 22:43:07.906,NaT,NaN,NaN,NaN
1,68d04b6b8255b4901067c399,card,0000419b-0bba-4488-8f7a-6194544ce91e,b34bb2dc-c1af-4d77-b0b3-a0fb342a5fc6,[668564],129825.0,91829.0,558404.0,777725.0,Forest,...,scryfall_usd_foil,558404.0,777725.0,2025-12-12,2025-12-12T22:43:07.906282+00:00,2025-12-12 22:43:07.906,NaT,NaN,NaN,NaN
2,68d04b6b8255b4901067c399,card,0000419b-0bba-4488-8f7a-6194544ce91e,b34bb2dc-c1af-4d77-b0b3-a0fb342a5fc6,[668564],129825.0,91829.0,558404.0,777725.0,Forest,...,NaN,558404.0,777725.0,2025-12-13,NaN,NaT,2025-12-12 23:05:29.334,NaN,NaN,NaN
3,68d04b6b8255b4901067c39a,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,NaN,14240.0,13850.0,Fury Sliver,...,scryfall_bulk,14240.0,13850.0,2025-12-13,NaN,NaT,2025-12-13 00:01:04.421,0.44,usd,nonfoil
4,68d04b6b8255b4901067c39a,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,NaN,14240.0,13850.0,Fury Sliver,...,scryfall_bulk,14240.0,13850.0,2025-12-13,NaN,NaT,2025-12-13 00:01:04.421,3.72,usd,foil


## Data Exploration & Cleaning

Explore the data structure and identify pricing columns, card types, and prepare for analysis.

In [6]:
# Explore the data structure
print("=== DATA EXPLORATION ===")
print(f"Total cards: {len(df)}")
print(f"DataFrame info:")
df.info()

print("\n=== COLUMN ANALYSIS ===")
print("Available columns:")
for i, col in enumerate(df.columns):
    print(f"{i+1:2d}. {col}")

# Look for pricing columns
pricing_columns = [col for col in df.columns if 'price' in col.lower() or 'usd' in col.lower()]
print(f"\nPricing columns found: {pricing_columns}")

# Look for type columns
type_columns = [col for col in df.columns if 'type' in col.lower()]
print(f"Type columns found: {type_columns}")

# Check for card name and other key fields
key_fields = ['name', 'id', 'set', 'rarity']
existing_fields = [col for col in key_fields if col in df.columns]
print(f"Key fields available: {existing_fields}")

print("\n=== SAMPLE DATA ===")
# Show a sample of the most relevant columns
sample_cols = ['name'] + type_columns + pricing_columns + ['rarity', 'set']
sample_cols = [col for col in sample_cols if col in df.columns][:8]  # Limit to first 8 relevant columns
df[sample_cols].head()

=== DATA EXPLORATION ===
Total cards: 321433
DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321433 entries, 0 to 321432
Columns: 108 entries, _id to finish
dtypes: bool(14), datetime64[ns](3), float64(13), object(78)
memory usage: 234.8+ MB

=== COLUMN ANALYSIS ===
Available columns:
 1. _id
 2. object
 3. id
 4. oracle_id
 5. multiverse_ids
 6. mtgo_id
 7. arena_id
 8. tcgplayer_id
 9. cardmarket_id
10. name
11. lang
12. released_at
13. uri
14. scryfall_uri
15. layout
16. highres_image
17. image_status
18. image_uris
19. mana_cost
20. cmc
21. type_line
22. oracle_text
23. colors
24. color_identity
25. keywords
26. produced_mana
27. legalities
28. games
29. reserved
30. game_changer
31. foil
32. nonfoil
33. finishes
34. oversized
35. promo
36. reprint
37. variation
38. set_id
39. set
40. set_name
41. set_type
42. set_uri
43. set_search_uri
44. scryfall_set_uri
45. rulings_uri
46. prints_search_uri
47. collector_number
48. digital
49. rarity
50. card_back_id
51. artis

,name,type_line,set_type,promo_types,printed_type_line,price_type,prices,last_price_update
0,Forest,Basic Land — Forest,expansion,NaN,NaN,usd,"{'usd': '0.24', 'usd_foil': '0.56', 'usd_etche...",2025-12-11 03:28:45.347
1,Forest,Basic Land — Forest,expansion,NaN,NaN,usd_foil,"{'usd': '0.24', 'usd_foil': '0.56', 'usd_etche...",2025-12-11 03:28:45.347
2,Forest,Basic Land — Forest,expansion,NaN,NaN,usd,"{'usd': '0.24', 'usd_foil': '0.56', 'usd_etche...",2025-12-11 03:28:45.347
3,Fury Sliver,Creature — Sliver,expansion,NaN,NaN,usd,"{'usd': '0.40', 'usd_foil': '3.79', 'usd_etche...",2025-12-11 03:28:45.405
4,Fury Sliver,Creature — Sliver,expansion,NaN,NaN,usd_foil,"{'usd': '0.40', 'usd_foil': '3.79', 'usd_etche...",2025-12-11 03:28:45.405


## Filter Out Basic Lands

Remove basic lands from the dataset to focus analysis on non-basic card types.

In [7]:
# Identify the type column (most likely 'type_line' or 'type')
type_col = None
for col in ['type_line', 'types', 'type', 'card_type']:
    if col in df.columns:
        type_col = col
        break

print(f"Using type column: {type_col}")

if type_col:
    # Check unique values in type column to understand the data
    print(f"\nSample type values:")
    print(df[type_col].value_counts().head(10))
    
    # Filter out basic lands
    # Basic lands typically have "Basic Land" in their type line
    initial_count = len(df)
    
    # Create filters for basic lands
    basic_land_filters = [
        df[type_col].str.contains('Basic Land', case=False, na=False),
        df[type_col].str.contains('Basic Snow Land', case=False, na=False),
        # Also filter specific basic land names
        df['name'].isin(['Plains', 'Island', 'Swamp', 'Mountain', 'Forest', 
                        'Snow-Covered Plains', 'Snow-Covered Island', 'Snow-Covered Swamp', 
                        'Snow-Covered Mountain', 'Snow-Covered Forest'])
    ]
    
    # Combine all basic land filters
    is_basic_land = pd.Series([False] * len(df))
    for filter_condition in basic_land_filters:
        is_basic_land = is_basic_land | filter_condition
    
    # Filter out basic lands
    df_filtered = df[~is_basic_land].copy()
    
    basic_lands_removed = initial_count - len(df_filtered)
    print(f"\nFiltering Results:")
    print(f"Initial card count: {initial_count:,}")
    print(f"Basic lands removed: {basic_lands_removed:,}")
    print(f"Remaining cards: {len(df_filtered):,}")
    print(f"Percentage remaining: {(len(df_filtered)/initial_count)*100:.1f}%")
    
else:
    print("Warning: Could not find type column. Proceeding with all cards.")
    df_filtered = df.copy()

Using type column: type_line

Sample type values:
type_line
Instant                     34014
Sorcery                     32249
Land                        16754
Enchantment                 16289
Artifact                    15110
Enchantment — Aura           9631
Artifact — Equipment         4627
Creature — Human Wizard      3593
Creature — Human Soldier     3158
Creature — Elemental         3102
Name: count, dtype: int64

Filtering Results:
Initial card count: 321,433
Basic lands removed: 12,705
Remaining cards: 308,728
Percentage remaining: 96.0%


## Prepare Pricing Data

Extract and clean pricing information, handling missing values and converting to numeric format.

In [13]:
# Debug: Let's examine what's in the pricing columns more closely
print("=== DEBUGGING PRICING DATA ===")

# Check the 'prices' column structure (which seems promising)
print("\nSample 'prices' column values:")
sample_prices = df_filtered['prices'].dropna().head(10)
for i, price_val in enumerate(sample_prices):
    print(f"Sample {i+1}: Type={type(price_val)}, Value={price_val}")

# Check price_usd column
print(f"\nprice_usd column stats:")
print(f"Total non-null values: {df_filtered['price_usd'].count()}")
print(f"Unique values: {df_filtered['price_usd'].nunique()}")
print(f"Sample values: {df_filtered['price_usd'].dropna().head(10).tolist()}")

# Check other pricing columns
for col in ['price_type', 'price_value']:
    if col in df_filtered.columns:
        print(f"\n{col} column stats:")
        print(f"Non-null values: {df_filtered[col].count()}")
        print(f"Sample values: {df_filtered[col].dropna().head(5).tolist()}")

print("\n" + "="*50)

=== DEBUGGING PRICING DATA ===

Sample 'prices' column values:
Sample 1: Type=<class 'dict'>, Value={'usd': '0.40', 'usd_foil': '3.79', 'usd_etched': None, 'eur': '0.19', 'eur_foil': '1.42', 'tix': '0.03'}
Sample 2: Type=<class 'dict'>, Value={'usd': '0.40', 'usd_foil': '3.79', 'usd_etched': None, 'eur': '0.19', 'eur_foil': '1.42', 'tix': '0.03'}
Sample 3: Type=<class 'dict'>, Value={'usd': '0.40', 'usd_foil': '3.79', 'usd_etched': None, 'eur': '0.19', 'eur_foil': '1.42', 'tix': '0.03'}
Sample 4: Type=<class 'dict'>, Value={'usd': '0.40', 'usd_foil': '3.79', 'usd_etched': None, 'eur': '0.19', 'eur_foil': '1.42', 'tix': '0.03'}
Sample 5: Type=<class 'dict'>, Value={'usd': '0.40', 'usd_foil': '3.79', 'usd_etched': None, 'eur': '0.19', 'eur_foil': '1.42', 'tix': '0.03'}
Sample 6: Type=<class 'dict'>, Value={'usd': '0.14', 'usd_foil': '1.69', 'usd_etched': None, 'eur': '0.27', 'eur_foil': '2.40', 'tix': '0.03'}
Sample 7: Type=<class 'dict'>, Value={'usd': '0.14', 'usd_foil': '1.69', 'usd_e

In [14]:
# IMPROVED PRICING EXTRACTION
print("=== IMPROVED PRICING DATA EXTRACTION ===")

df_analysis = df_filtered.copy()

# Strategy 1: Extract from 'prices' dictionary column
def extract_usd_from_prices_dict(prices_dict):
    if pd.isna(prices_dict) or not isinstance(prices_dict, dict):
        return np.nan
    usd_val = prices_dict.get('usd', np.nan)
    if pd.isna(usd_val) or usd_val is None:
        return np.nan
    try:
        return float(usd_val)
    except:
        return np.nan

# Strategy 2: Extract from price_type/price_value columns
def extract_usd_from_type_value():
    # Create a mask for USD prices
    usd_mask = df_analysis['price_type'] == 'usd'
    return df_analysis.loc[usd_mask, 'price_value'].astype(float)

# Apply Strategy 1: Extract from prices dictionary
df_analysis['price_from_dict'] = df_analysis['prices'].apply(extract_usd_from_prices_dict)

# Apply Strategy 2: Extract from type/value structure
# Create a series to hold USD prices from the type/value structure
price_from_type_value = pd.Series(index=df_analysis.index, dtype=float)
if 'price_type' in df_analysis.columns and 'price_value' in df_analysis.columns:
    usd_indices = df_analysis[df_analysis['price_type'] == 'usd'].index
    price_from_type_value.loc[usd_indices] = df_analysis.loc[usd_indices, 'price_value'].astype(float)

# Strategy 3: Use existing price_usd if available
price_from_usd_col = df_analysis['price_usd'] if 'price_usd' in df_analysis.columns else pd.Series(index=df_analysis.index, dtype=float)

# Combine all strategies - use the first available price
df_analysis['price_clean'] = df_analysis['price_from_dict'].fillna(price_from_type_value).fillna(price_from_usd_col)

# Remove invalid prices
initial_count = len(df_analysis)
df_analysis = df_analysis.dropna(subset=['price_clean'])
df_analysis = df_analysis[df_analysis['price_clean'] > 0]  # Remove zero or negative prices

print(f"\nIMPROVED Pricing Data Results:")
print(f"Cards before pricing filter: {initial_count:,}")
print(f"Cards with valid pricing: {len(df_analysis):,}")
print(f"Cards removed (no/invalid pricing): {initial_count - len(df_analysis):,}")
print(f"Success rate: {len(df_analysis)/initial_count*100:.1f}%")

# Pricing method breakdown
has_dict_price = df_analysis['price_from_dict'].notna().sum()
has_type_value_price = price_from_type_value.loc[df_analysis.index].notna().sum()
has_usd_col_price = price_from_usd_col.loc[df_analysis.index].notna().sum()

print(f"\nPricing source breakdown:")
print(f"From 'prices' dict: {has_dict_price:,}")
print(f"From type/value structure: {has_type_value_price:,}")
print(f"From 'price_usd' column: {has_usd_col_price:,}")

# Price statistics
print(f"\nPrice Range (Improved):")
print(f"Min price: ${df_analysis['price_clean'].min():.2f}")
print(f"Max price: ${df_analysis['price_clean'].max():.2f}")
print(f"Median price: ${df_analysis['price_clean'].median():.2f}")
print(f"Mean price: ${df_analysis['price_clean'].mean():.2f}")

print("=" * 50)

=== IMPROVED PRICING DATA EXTRACTION ===

IMPROVED Pricing Data Results:
Cards before pricing filter: 308,728
Cards with valid pricing: 280,092
Cards removed (no/invalid pricing): 28,636
Success rate: 90.7%

Pricing source breakdown:
From 'prices' dict: 279,579
From type/value structure: 77,139
From 'price_usd' column: 727

Price Range (Improved):
Min price: $0.01
Max price: $3418.22
Median price: $0.24
Mean price: $3.30


## Extract Card Types

Parse the type line to extract primary card types for grouping analysis.

In [16]:
def extract_primary_card_type(type_line):
    """Extract the primary card type from a type line"""
    if pd.isna(type_line):
        return 'Unknown'
    
    type_str = str(type_line).lower()
    
    # Define card type priorities (more specific first)
    card_types = {
        'planeswalker': ['planeswalker'],
        'creature': ['creature'],
        'artifact_creature': ['artifact creature', 'artifact — creature'],
        'enchantment_creature': ['enchantment creature', 'enchantment — creature'], 
        'instant': ['instant'],
        'sorcery': ['sorcery'],
        'enchantment': ['enchantment'],
        'artifact': ['artifact'],
        'land': ['land'],  # Non-basic lands
        'battle': ['battle'],
        'tribal': ['tribal'],
        'conspiracy': ['conspiracy']
    }
    
    # Check for each type (order matters for compound types)
    for card_type, patterns in card_types.items():
        for pattern in patterns:
            if pattern in type_str:
                return card_type.replace('_', ' ').title()
    
    return 'Other'

# Apply type extraction
df_analysis['primary_type'] = df_analysis[type_col].apply(extract_primary_card_type)

# Check type distribution
print("Primary Type Distribution:")
type_counts = df_analysis['primary_type'].value_counts()
for card_type, count in type_counts.items():
    percentage = (count / len(df_analysis)) * 100
    print(f"  {card_type:<20}: {count:>7,} cards ({percentage:>5.1f}%)")

print(f"\nTotal unique types identified: {len(type_counts)}")
print(f"Cards successfully categorized: {len(df_analysis):,}")

# Show some examples
print(f"\nSample type categorizations:")
sample_types = df_analysis[[type_col, 'primary_type', 'name']].dropna().drop_duplicates('primary_type').head(10)
for _, row in sample_types.iterrows():
    name_str = str(row['name'])[:25] if pd.notna(row['name']) else 'Unknown'
    type_str = str(row[type_col])[:30] if pd.notna(row[type_col]) else 'Unknown'
    print(f"  {name_str:<25} | {type_str:<30} | {row['primary_type']}")

Primary Type Distribution:
  Creature            : 141,868 cards ( 50.7%)
  Instant             :  33,602 cards ( 12.0%)
  Sorcery             :  30,848 cards ( 11.0%)
  Enchantment         :  26,863 cards (  9.6%)
  Artifact            :  22,830 cards (  8.2%)
  Land                :  19,220 cards (  6.9%)
  Planeswalker        :   3,362 cards (  1.2%)
  Other               :   1,184 cards (  0.4%)
  Unknown             :     210 cards (  0.1%)
  Conspiracy          :     105 cards (  0.0%)

Total unique types identified: 10
Cards successfully categorized: 280,092

Sample type categorizations:
  Fury Sliver               | Creature — Sliver              | Creature
  Web                       | Enchantment — Aura             | Enchantment
  Wastewood Verge           | Land                           | Land
  Surge of Brilliance       | Instant                        | Instant
  Wildcall                  | Sorcery                        | Sorcery
  Fell Beast's Shriek // Fe | Card // Car

## Calculate Comprehensive Statistics by Card Type

Group data by card type and calculate detailed statistics including count, min, max, mean, median, and percentiles.

In [17]:
# Calculate comprehensive statistics by card type
def calculate_price_statistics(group):
    """Calculate comprehensive price statistics for a group"""
    prices = group['price_clean']
    
    return pd.Series({
        'count': len(prices),
        'min_price': prices.min(),
        'max_price': prices.max(), 
        'mean_price': prices.mean(),
        'median_price': prices.median(),
        'std_price': prices.std(),
        'q25_price': prices.quantile(0.25),
        'q75_price': prices.quantile(0.75),
        'q90_price': prices.quantile(0.90),
        'q95_price': prices.quantile(0.95),
        'q99_price': prices.quantile(0.99)
    })

# Group by primary type and calculate statistics
print("Calculating statistics by card type...")
stats_by_type = df_analysis.groupby('primary_type').apply(calculate_price_statistics, include_groups=False)

# Sort by count (most common types first)
stats_by_type = stats_by_type.sort_values('count', ascending=False)

print(f"Statistics calculated for {len(stats_by_type)} card types")
print(f"Based on {df_analysis['count'].sum() if 'count' in df_analysis.columns else len(df_analysis)} total cards")

# Preview the statistics
stats_by_type.head()

Calculating statistics by card type...
Statistics calculated for 10 card types
Based on 280092 total cards


,count,min_price,max_price,mean_price,median_price,std_price,q25_price,q75_price,q90_price,q95_price,q99_price
primary_type,,,,,,,,,,,
Creature,141868.0,0.01,3418.22,2.151897,0.21,19.447895,0.09,0.68,3.590,8.2200,32.6700
Instant,33602.0,0.01,2968.93,2.570665,0.18,27.275866,0.08,0.56,3.279,8.3200,44.8574
Sorcery,30848.0,0.01,2650.00,2.848330,0.25,31.503408,0.11,0.81,3.670,8.0300,33.6400
Enchantment,26863.0,0.01,1450.00,3.507712,0.33,23.530722,0.14,1.56,5.938,14.0000,41.8900
Artifact,22830.0,0.01,3290.00,7.735823,0.42,72.975639,0.17,2.34,9.141,20.4465,90.0600


## Display Comprehensive Statistical Summary Table

Create a beautifully formatted table showing all calculated statistics organized by card type.

In [18]:
# Create a comprehensive summary table
summary_table = stats_by_type.copy()

# Round monetary values to 2 decimal places
price_columns = ['min_price', 'max_price', 'mean_price', 'median_price', 'std_price', 
                'q25_price', 'q75_price', 'q90_price', 'q95_price', 'q99_price']

for col in price_columns:
    summary_table[col] = summary_table[col].round(2)

# Format for display
display_table = summary_table[['count', 'min_price', 'q25_price', 'median_price', 
                              'mean_price', 'q75_price', 'q90_price', 'max_price', 'std_price']].copy()

# Rename columns for better readability
display_table.columns = ['Count', 'Min $', '25th %', 'Median $', 'Mean $', '75th %', '90th %', 'Max $', 'Std Dev']

# Format count column as integers
display_table['Count'] = display_table['Count'].astype(int)

print("="*100)
print("MTG CARD PRICING STATISTICS BY TYPE")
print("="*100)
print("(Excludes Basic Lands)")
print("")

# Display the table with nice formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '${:.2f}'.format)

print(display_table.to_string())

print(f"\n{'='*100}")
print(f"SUMMARY")
print(f"{'='*100}")
print(f"Total card types analyzed: {len(display_table)}")
print(f"Total cards with pricing: {display_table['Count'].sum():,}")
print(f"Overall price range: ${summary_table['min_price'].min():.2f} - ${summary_table['max_price'].max():.2f}")
print(f"Dataset median price: ${df_analysis['price_clean'].median():.2f}")
print(f"Dataset mean price: ${df_analysis['price_clean'].mean():.2f}")

# Reset display options
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.float_format')

MTG CARD PRICING STATISTICS BY TYPE
(Excludes Basic Lands)

               Count  Min $  25th %  Median $  Mean $  75th %  90th %    Max $  Std Dev
primary_type                                                                           
Creature      141868  $0.01   $0.09     $0.21   $2.15   $0.68   $3.59 $3418.22   $19.45
Instant        33602  $0.01   $0.08     $0.18   $2.57   $0.56   $3.28 $2968.93   $27.28
Sorcery        30848  $0.01   $0.11     $0.25   $2.85   $0.81   $3.67 $2650.00   $31.50
Enchantment    26863  $0.01   $0.14     $0.33   $3.51   $1.56   $5.94 $1450.00   $23.53
Artifact       22830  $0.01   $0.17     $0.42   $7.74   $2.34   $9.14 $3290.00   $72.98
Land           19220  $0.03   $0.18     $0.38   $7.92   $2.72  $13.35 $2700.00   $64.81
Planeswalker    3362  $0.12   $0.71     $1.98   $4.73   $4.63  $10.35  $354.47   $13.27
Other           1184  $0.04   $0.32     $1.39   $4.07   $4.03   $9.25  $154.97    $9.89
Unknown          210  $0.86   $2.22     $5.38   $9.41  $11.8

## Additional Analysis: Price Distribution Insights

Examine interesting patterns and insights from the pricing data by card type.

In [19]:
# Additional insights and analysis
print("="*80)
print("PRICE DISTRIBUTION INSIGHTS")
print("="*80)

# Most expensive card by type
print("\n🔥 MOST EXPENSIVE CARDS BY TYPE:")
for card_type in stats_by_type.head(8).index:  # Top 8 most common types
    type_cards = df_analysis[df_analysis['primary_type'] == card_type]
    most_expensive = type_cards.loc[type_cards['price_clean'].idxmax()]
    print(f"  {card_type:<15}: ${most_expensive['price_clean']:>7.2f} - {most_expensive['name']}")

# Types with highest average prices
print(f"\n💰 HIGHEST AVERAGE PRICES BY TYPE:")
high_value_types = stats_by_type.sort_values('mean_price', ascending=False).head(6)
for card_type, stats in high_value_types.iterrows():
    if stats['count'] >= 10:  # Only include types with reasonable sample size
        print(f"  {card_type:<15}: ${stats['mean_price']:>7.2f} avg (n={stats['count']:,})")

# Price volatility (highest standard deviation)
print(f"\n📈 MOST VOLATILE PRICING (High Std Dev):")
volatile_types = stats_by_type.sort_values('std_price', ascending=False).head(6)
for card_type, stats in volatile_types.iterrows():
    if stats['count'] >= 10:
        cv = stats['std_price'] / stats['mean_price']  # Coefficient of variation
        print(f"  {card_type:<15}: ${stats['std_price']:>7.2f} std dev, CV: {cv:.2f}")

# Types with most cards
print(f"\n📊 LARGEST CARD TYPE CATEGORIES:")
for card_type, stats in stats_by_type.head(5).iterrows():
    percentage = (stats['count'] / display_table['Count'].sum()) * 100
    print(f"  {card_type:<15}: {stats['count']:>7,} cards ({percentage:>4.1f}% of dataset)")

print(f"\n{'='*80}")

# Close database connection
client.close()
print("✅ Database connection closed.")
print("Analysis complete! 🎉")
print(f"Notebook location: /workspaces/mtgecorec/notebooks/card_pricing_analysis.ipynb")

PRICE DISTRIBUTION INSIGHTS

🔥 MOST EXPENSIVE CARDS BY TYPE:
  Creature       : $3418.22 - Cloud, Midgar Mercenary
  Instant        : $2968.93 - Ancestral Recall
  Sorcery        : $2650.00 - Time Walk
  Enchantment    : $1450.00 - Raging River
  Artifact       : $3290.00 - Mox Sapphire
  Land           : $2700.00 - The Tabernacle at Pendrell Vale
  Planeswalker   : $ 354.47 - Jace, the Mind Sculptor
  Other          : $ 154.97 - Titania

💰 HIGHEST AVERAGE PRICES BY TYPE:
  Unknown        : $   9.41 avg (n=210.0)
  Land           : $   7.92 avg (n=19,220.0)
  Artifact       : $   7.74 avg (n=22,830.0)
  Planeswalker   : $   4.73 avg (n=3,362.0)
  Other          : $   4.07 avg (n=1,184.0)
  Enchantment    : $   3.51 avg (n=26,863.0)

📈 MOST VOLATILE PRICING (High Std Dev):
  Artifact       : $  72.98 std dev, CV: 9.43
  Land           : $  64.81 std dev, CV: 8.19
  Sorcery        : $  31.50 std dev, CV: 11.06
  Instant        : $  27.28 std dev, CV: 10.61
  Enchantment    : $  23.53 std

## Quick Check: Today's Pricing Collection Status

Simple count of today's pricing records and unique cards processed.

In [13]:
# Quick status check for today's pricing collection
from datetime import date

# Get today's date
today = date.today().isoformat()
print(f"📊 Pricing Collection Status for {today}")
print("=" * 50)

# Get fresh database connection
client = get_mongo_client()
pricing_collection = get_collection(client, "mtgecorec", "card_pricing_daily")
cards_collection = get_collection(client, "mtgecorec", "cards")

# Count total records for today
total_records_today = pricing_collection.count_documents({'date': today})
print(f"Total pricing records for today: {total_records_today:,}")

# Count unique cards with pricing for today
unique_cards_today = len(set(pricing_collection.distinct('scryfall_id', {'date': today})))
print(f"Unique cards with pricing for today: {unique_cards_today:,}")

# Get total cards in database for comparison
total_cards = cards_collection.count_documents({})
print(f"Total cards in database: {total_cards:,}")

# Calculate coverage and remaining
coverage_pct = (unique_cards_today / total_cards) * 100 if total_cards > 0 else 0
remaining_cards = total_cards - unique_cards_today

print(f"Coverage: {coverage_pct:.1f}%")
print(f"Cards remaining: {remaining_cards:,}")

# Status check
if remaining_cards < 1000:
    print("✅ COLLECTION COMPLETE!")
else:
    print("⚠️  Collection still in progress")
    
# Close connection
client.close()

📊 Pricing Collection Status for 2025-12-25
Total pricing records for today: 57,618
Unique cards with pricing for today: 17,922
Total cards in database: 110,031
Coverage: 16.3%
Cards remaining: 92,109
⚠️  Collection still in progress


In [12]:
# Reset stuck pricing lock and trigger new collection
import requests
import json

print("🔧 Azure Functions Management")
print("=" * 40)

# Correct Azure Functions URL
base_url = "https://mtgecorecfunc-akeuc0excwg9h7dd.westus3-01.azurewebsites.net"

# Step 1: Reset the lock
print("1. Resetting pricing collection lock...")
try:
    reset_response = requests.post(
        f"{base_url}/api/pricing/reset_lock", 
        timeout=30
    )
    if reset_response.status_code == 200:
        print("✅ Lock reset successfully")
        reset_data = reset_response.json()
        print(f"   Was running: {reset_data.get('was_running', 'Unknown')}")
    else:
        print(f"❌ Lock reset failed: {reset_response.status_code}")
        print(f"Response: {reset_response.text[:200]}...")
except Exception as e:
    print(f"❌ Lock reset error: {e}")

print()

# Step 2: Trigger new collection with fixed auto-chaining
print("2. Triggering pricing collection with auto-chaining...")
try:
    trigger_response = requests.get(
        f"{base_url}/api/pricing/collect",
        timeout=30
    )
    if trigger_response.status_code == 200:
        print("✅ Collection triggered successfully")
        trigger_data = trigger_response.json()
        print(f"   Status: {trigger_data.get('status', 'Unknown')}")
        if 'batch_info' in trigger_data:
            batch_info = trigger_data['batch_info']
            print(f"   Batch size: {batch_info.get('batch_size', 'Unknown'):,}")
            print(f"   Auto-continue: {batch_info.get('auto_continue', 'Unknown')}")
    else:
        print(f"❌ Collection trigger failed: {trigger_response.status_code}")
        print(f"Response: {trigger_response.text[:200]}...")
except Exception as e:
    print(f"❌ Collection trigger error: {e}")

print("\n🚀 Check Azure Functions logs to monitor progress!")
print("Expected: Should process all 92k+ remaining cards with auto-chaining")

🔧 Azure Functions Management
1. Resetting pricing collection lock...
✅ Lock reset successfully
   Was running: False

2. Triggering pricing collection with auto-chaining...
❌ Collection trigger error: HTTPSConnectionPool(host='mtgecorecfunc-akeuc0excwg9h7dd.westus3-01.azurewebsites.net', port=443): Read timed out. (read timeout=30)

🚀 Check Azure Functions logs to monitor progress!
Expected: Should process all 92k+ remaining cards with auto-chaining
